In [72]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report


create a Price Range Prediction model using Random Forest regression 
Predict the price range of restaurants based on features such as review count, rating.
This model can help users filter restaurants by price range, allowing them to find the options that match their budget.

In [37]:
# Load the data into a pandas DataFrame
data = pd.read_csv('cleaned_data.csv') 
data.head(3)

,id,name,image_url,is_closed,url,review_count,rating,transactions,price,group_city,cuisines,latitude,longitude,state
0,a0IET3_yCFcO36OqGSsisg,Eataly NYC Flatiron,https://s3-media4.fl.yelpcdn.com/bphoto/1UDlnu...,False,https://www.yelp.com/biz/eataly-nyc-flatiron-n...,6101,4.0,"delivery, pickup",2,New York City,Italian,40.742101,-73.989922,NY
1,zj8Lq1T8KIC5zwFief15jg,Prince Street Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/PfI8oV...,False,https://www.yelp.com/biz/prince-street-pizza-n...,5028,4.5,"delivery, pickup",1,New York City,Italian,40.723088,-73.994530,NY
2,16ZnHpuaaBt92XWeJHCC5A,Olio e Più,https://s3-media4.fl.yelpcdn.com/bphoto/CUpPgz...,False,https://www.yelp.com/biz/olio-e-pi%C3%B9-new-y...,4828,4.5,"pickup, delivery",2,New York City,Italian,40.733798,-73.999774,NY


In [38]:
# Check for missing values in the 'review_count' column
missing_review_count = data['review_count'].isnull().sum()

# Check for missing values in the 'rating' column
missing_rating = data['rating'].isnull().sum()

# Check for missing values in the 'price' column
missing_price = data['price'].isnull().sum()

print(f'Missing review_count values: {missing_review_count}')
print(f'Missing rating values: {missing_rating}')
print(f'Missing price values: {missing_price}')

Missing review_count values: 0
Missing rating values: 0
Missing price values: 0


In [39]:
# Drop unwanted columns
data.drop(["id", "image_url", "url"], axis=1, inplace=True)
data.head(3)

,name,is_closed,review_count,rating,transactions,price,group_city,cuisines,latitude,longitude,state
0,Eataly NYC Flatiron,False,6101,4.0,"delivery, pickup",2,New York City,Italian,40.742101,-73.989922,NY
1,Prince Street Pizza,False,5028,4.5,"delivery, pickup",1,New York City,Italian,40.723088,-73.994530,NY
2,Olio e Più,False,4828,4.5,"pickup, delivery",2,New York City,Italian,40.733798,-73.999774,NY


In [41]:
# Split the data into training and testing sets
X = data.drop("price", axis=1)
y = data["price"]

In [42]:
X.head()

,name,is_closed,review_count,rating,transactions,group_city,cuisines,latitude,longitude,state
0,Eataly NYC Flatiron,False,6101,4.0,"delivery, pickup",New York City,Italian,40.742101,-73.989922,NY
1,Prince Street Pizza,False,5028,4.5,"delivery, pickup",New York City,Italian,40.723088,-73.994530,NY
2,Olio e Più,False,4828,4.5,"pickup, delivery",New York City,Italian,40.733798,-73.999774,NY
3,L & B Spumoni Gardens,False,4648,4.0,"delivery, pickup",New York City,Italian,40.594715,-73.981316,NY
4,Carmine's Italian Restaurant - Times Square,False,4640,4.0,"delivery, pickup",New York City,Italian,40.757498,-73.986653,NY


In [43]:
# Preview the first five entries for the target variable
y[:5]

0    2
1    1
2    2
3    2
4    2
Name: price, dtype: int64

In [46]:
# Encode the categorical variables using get_dummies
X = pd.get_dummies(X)

In [48]:
# Review the features data
X.head()

,is_closed,review_count,rating,latitude,longitude,name_ Cinco De Mayo Mexican Restaurant Murfreesboro,name_100 South Chophouse,name_100% Taquito,name_100% de Agave,name_12 South Bistro,...,state_ON,state_OR,state_PA,state_PE,state_SC,state_TN,state_TX,state_VA,state_WA,state_WI
0,False,6101,4.0,40.742101,-73.989922,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,False,5028,4.5,40.723088,-73.994530,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,False,4828,4.5,40.733798,-73.999774,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,False,4648,4.0,40.594715,-73.981316,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,False,4640,4.0,40.757498,-73.986653,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [51]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [52]:
# Remove rows with missing values from X_train and y_train
X_train.dropna(inplace=True)
y_train = y_train[X_train.index]

# Remove corresponding rows with missing values from X_test and y_test
X_test.dropna(inplace=True)
y_test = y_test[X_test.index]

In [57]:
# Create the StandardScaler instance
scaler = StandardScaler()

In [58]:
# Fit the Standard Scaler with the training data
X_scaler = scaler.fit(X_train)

In [59]:
# Scale the training data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [65]:
# Create the random forest classifier instance
rf_model = RandomForestClassifier(n_estimators=42, random_state=100)


In [66]:
# Fit the model and use .ravel()on the "y_train" data. 
rf_model = rf_model.fit(X_train_scaled, y_train.ravel())

In [68]:
X.shape

(19145, 13362)

In [69]:
y.shape

(19145,)

In [70]:
# Making predictions using the testing data
predictions = rf_model.predict(X_test_scaled)

In [75]:
# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1", "Actual 2", "Actual 3", "Actual 4"], columns=["Actual 0", "Actual 1", "Actual 2", "Actual 3", "Actual 4"]
)

# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions)


In [76]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Actual 0,Actual 1,Actual 2,Actual 3,Actual 4
Actual 0,291,129,86,0,0
Actual 1,104,637,408,0,0
Actual 2,46,196,1766,4,0
Actual 3,1,3,144,4,0
Actual 4,0,0,8,2,0


Accuracy Score : 0.7046226168712457
Classification Report
              precision    recall  f1-score   support

           0       0.66      0.58      0.61       506
           1       0.66      0.55      0.60      1149
           2       0.73      0.88      0.80      2012
           3       0.40      0.03      0.05       152
           4       0.00      0.00      0.00        10

    accuracy                           0.70      3829
   macro avg       0.49      0.41      0.41      3829
weighted avg       0.69      0.70      0.68      3829



c:\Users\dgnil\anaconda3\envs\dev\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\dgnil\anaconda3\envs\dev\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\dgnil\anaconda3\envs\dev\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [25]:
new_data = pd.DataFrame({
    'review_count': [5028],
    'rating': [4.5]
})
predicted_price = model.predict(new_data)
print(f"Predicted Price Range: {predicted_price[0]}")

Predicted Price Range: 1.42
